### Setup

In [ ]:
!pip install torchprofile 1>/dev/null

In [ ]:
import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List

import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm

from torchprofile import profile_macs

import torch.nn.functional as F

assert torch.cuda.is_available(), \
"The current runtime does not have CUDA support." \
"Please go to menu bar (Runtime - Change runtime type) and select GPU"

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
def train(
    model: nn.Module,
    dataloader:DataLoader,
    criterion:nn.Module,
    optimizer:Optimizer,
    scheduler:StepLR,
    callbacks=None
) -> None:
  # 设置为训练模式
  model.train()

  for inputs,targets in tqdm(dataloader,desc='train',leave=False):
    # 将数据移动到GPU
    inputs=inputs.cuda()
    targets=targets.cuda()

    # 梯度清零
    optimizer.zero_grad()

    # 前向推理
    outputs=model(inputs)
    loss=criterion(outputs,targets)

    # 反向传播
    loss.backward()

    # 更新优化器
    optimizer.step()

    if callbacks is not None:
      for callback in callbacks:
        callback()

  # 更新调度器
  scheduler.step()

In [ ]:
@torch.inference_mode()
def evaluate(
    model:nn.Module,
    dataloader:DataLoader,
    verbose=True,
) -> float:
  model.eval()

  num_samples=0
  num_correct=0

  for inputs,targets in tqdm(dataloader,desc='eval',leave=False,disable=not verbose):
    inputs=inputs.cuda()
    targets=targets.cuda()

    outputs=model(inputs)

    outputs=outputs.argmax(dim=-1)

    num_samples+=targets.size(0)
    num_correct+=(outputs == targets).sum()

  return (num_correct / num_samples*100).item()

Helper Functions

In [ ]:
def get_model_macs(model,inputs) -> int:
  return profile_macs(model,inputs)

def get_sparsity(tensor:torch.Tensor) -> float:
  return 1- float(tensor.count_nonzero()) / tensor.numel()

def get_model_sparsity(model: nn.Module) -> float:
  num_nonzeros,num_elements=0,0
  for param in model.parameters():
    num_nonzeros+=param.count_nonzero()
    num_elements+=param.numel()
  return 1 - float(num_nonzeros) / num_elements

def get_num_parameters(model: nn.Module,count_nonzero_only=False) -> int:
  num_counted_elements=0
  for param in model.parameters():
    if count_nonzero_only:
      num_counted_elements+=param.count_nonzero()
    else:
      num_counted_elements+=param.numel()
  return num_counted_elements

def get_model_size(model: nn.Module,data_width=32,count_nonzero_only=False) -> int:
  return get_num_parameters(model,count_nonzero_only) *data_width

Byte=8
KiB=1024*Byte
MiB=1024*KiB
GiB=1024*MiB

Define prun functions

In [ ]:
# 逐元素判断 非结构化剪枝
def fine_grained_prune(tensor: torch.Tensor, sparsity: float) -> torch.Tensor:
  # 确保稀疏度在0-1之间
  sparsity=min(max(0.0,sparsity),1.0)
  if sparsity == 1.0:
    tensor.zero_()
    return torch.zeros_like(tensor)
  elif sparsity == 0.0:
    return torch.ones_like(tensor)

  num_elements=tensor.numel()

  num_zeros=round(num_elements*sparsity)
  importance=tensor.abs()
  threshold=importance.view(-1).kthvalue(num_zeros).values
  mask=torch.gt(importance,threshold)
  tensor.mul_(mask)

  return mask

In [ ]:
class FineGrainedPruner:
  def __init__(self,model,sparsity_dict):
    self.masks=FineGrainedPruner.prune(model,sparsity_dict)

  # 禁用梯度计算
  @torch.no_grad()
  def apply(self,model):
    for name,param in model.named_parameters():
      if name in self.masks:
        param*=self.masks[name]

  @staticmethod
  @torch.no_grad()
  def prune(model,sparsity_dict):
    masks=dict()
    for name,param in model.named_parameters():
      # 只处理多维参数（权重）
      if param.dim()>1:
        # 同意裁剪 / 分层裁剪
        if isinstance(sparsity_dict,dict):
          masks[name]=fine_grained_prune(param,sparsity_dict[name])
        else:
          assert(sparsity_dict < 1 and sparsity_dict >= 0)
          if sparsity_dict > 0:
            masks[name]=fine_grained_prune(param,sparsity_dict)
    return masks

Load the MNIST dataset

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
# 将标准化后的张量转换为图片
to_image = lambda t: (t*0.3081+0.1307).squeeze(0).to('cpu').numpy()

dataset = {}
for split in ["train", "test"]:
  dataset[split] = MNIST(
    "data",
    train=(split == "train"),
    download=(split == "train"),
    transform=transform,
  )

dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=256 if split == 'train' else 1000,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True
  )

In [ ]:
demos = {0: 3, 1: 2, 2: 1, 3: 30, 4: 4, 5: 15, 6: 11, 7: 0, 8: 61, 9: 9}
demo_inputs, demo_images = [], []
# 对比模型预测输出和对应标签
for digit, index in demos.items():
    demo_inputs.append(copy.deepcopy(dataset['test'][index][0]))
    demo_images.append(to_image(demo_inputs[-1]))
demo_inputs = torch.stack(demo_inputs).cuda()

### Neural Network Model

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1=nn.Conv2d(1,32,3,1)
    self.conv2=nn.Conv2d(32,64,3,1)
    self.dropout1=nn.Dropout(0.25)
    self.dropout2=nn.Dropout(0.5)
    self.fc1=nn.Linear(9216,128)
    self.fc2=nn.Linear(128,10)

  def forward(self,x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output
model=Net().cuda()

### Visualize the Demo Images

In [ ]:
def visualize(with_predictions=False):
    plt.figure(figsize=(20, 10))
    predictions = model(demo_inputs).argmax(dim=1) if with_predictions else None
    for digit, index in demos.items():
        plt.subplot(1, 10, digit + 1)
        plt.imshow(demo_images[digit])
        if predictions is None:
            plt.title(f"digit: {digit}")
        else:
            plt.title(f"digit: {digit}\npred: {int(predictions[digit])}")
        plt.axis('off')
    plt.show()

visualize()

### Pre-train Neural Network on MNIST

In [ ]:
lr = 1.0
lr_step_gamma = 0.7
num_epochs = 5

optimizer = Adadelta(model.parameters(), lr=lr)
criterion = F.nll_loss
scheduler = StepLR(optimizer, step_size=1, gamma=lr_step_gamma)

best_accuracy = 0
best_checkpoint = dict()
for epoch in range(num_epochs):
    train(model, dataloader['train'], criterion, optimizer, scheduler)
    accuracy = evaluate(model, dataloader['test'])
    is_best = accuracy > best_accuracy
    if is_best:
        best_checkpoint['state_dict'] = copy.deepcopy(model.state_dict())
        best_accuracy = accuracy
    print(f'    Epoch {epoch+1:>2d} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%')

print(f"=> loading best checkpoint")
model.load_state_dict(best_checkpoint['state_dict'])
recover_model = lambda: model.load_state_dict(best_checkpoint['state_dict'])

In [ ]:
dense_model_accuracy = evaluate(model, dataloader['test'])
dense_model_size = get_model_size(model)
print(f"dense model has accuracy={dense_model_accuracy:.2f}%")
print(f"dense model has size={dense_model_size/MiB:.2f} MiB")
visualize(True)

In [ ]:
sparsity = 0.99
# 加载最佳权重
recover_model()
pruner = FineGrainedPruner(model, sparsity)
pruner.apply(model)
sparse_model_accuracy = evaluate(model, dataloader['test'])
sparse_model_size = get_model_size(model, count_nonzero_only=True)
print(f"{sparsity*100}% sparse model has accuracy={sparse_model_accuracy:.2f}%")
print(f"{sparsity*100}% sparse model has size={sparse_model_size/MiB:.2f} MiB, "
      f"which is {dense_model_size/sparse_model_size:.2f}X smaller than "
      f"the {dense_model_size/MiB:.2f} MiB dense model")
visualize(True)

In [ ]:
num_finetune_epochs = 2
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_finetune_epochs)

best_sparse_checkpoint = dict()
best_sparse_accuracy = 0
print(f'微调剪枝后的模型')
for epoch in range(num_finetune_epochs):

    train(model, dataloader['train'], criterion, optimizer, scheduler,
          callbacks=[lambda: pruner.apply(model)])
    accuracy = evaluate(model, dataloader['test'])
    is_best = accuracy > best_sparse_accuracy
    if is_best:
        best_sparse_checkpoint['state_dict'] = copy.deepcopy(model.state_dict())
        best_sparse_accuracy = accuracy
    print(f'    Epoch {epoch+1} Sparse Accuracy {accuracy:.2f}% / Best Sparse Accuracy: {best_sparse_accuracy:.2f}%')



In [ ]:
model.load_state_dict(best_sparse_checkpoint['state_dict'])
sparse_model_accuracy = evaluate(model, dataloader['test'])
sparse_model_size = get_model_size(model, count_nonzero_only=True)
print(f"{sparsity*100}% sparse model has accuracy={sparse_model_accuracy:.2f}%")
print(f"{sparsity*100}% sparse model has size={sparse_model_size/MiB:.2f} MiB, "
      f"which is {dense_model_size/sparse_model_size:.2f}X smaller than "
      f"the {dense_model_size/MiB:.2f} MiB dense model")
visualize(True)